In [1]:
# Import necessary libraries
import numpy as np
import torch
from torch import nn
from torch.nn import Transformer
from torch.utils.data import DataLoader, TensorDataset
import pickle

## NanoGPS adds
from torch.nn import functional as F

In [2]:
print("CUDA Available: ",torch.cuda.is_available())
use_cuda=True
device = torch.device("cuda:0" if use_cuda and torch.cuda.is_available() else "cpu")
print("device", device)

CUDA Available:  True
device cuda:0


In [4]:
%run create_trials.py "AB","AC","AD","AE","AF","AG"

Trainig pairs for the experiment ['AB', 'AC', 'AD', 'AE', 'AF', 'AG']
Creating experiment with 7 members, 4 classes and 21 dummy stimulus.
Experiment trials created!


In [6]:
train_info.sample(20)

,sample_subset,st_sample,st_comp1,st_comp2,st_comp3,option_answer,st_comparison
18466,train,A3,F2,E3,E1,O_2,E3
10263,train,A1,D1,D2,E3,O_1,D1
17470,train,A2,G1,E2,D4,O_2,E2
24357,train,A4,F4,B3,G1,O_1,F4
7880,train,A3,D4,B1,C3,O_3,C3
3547,train,A3,F1,B3,A2,O_2,B3
16554,train,A2,E2,C1,F4,O_1,E2
16619,train,A2,D4,C4,E2,O_3,E2
8101,train,A3,D1,C3,A4,O_2,C3
19888,train,A4,G2,E4,A3,O_2,E4


In [5]:
trials_info_columns=["st_sample","st_comp1","st_comp2","st_comp3","option_answer"]#"st_comparison"

train_trials_corpus_df=train_info[trials_info_columns]
train_dummy_trials_corpus_df=train_dummy_info[trials_info_columns]#"st_comparison"

In [7]:
# tokens_list=list(stims_random.keys())
tokens_list=sorted(list(set((list(np.array(train_trials_corpus_df).reshape([1,-1])[0])+(list(np.array(train_dummy_trials_corpus_df).reshape([1,-1])[0]))))))

In [8]:
token_to_index = {token: index for index, token in enumerate(tokens_list)}

In [9]:
# encode corpus in positional tokens train data class member comparison
encoded_training_data = np.array([[token_to_index[token] for token in sequence] for sequence in np.array(train_trials_corpus_df)])
train_tensor_encoded=torch.from_numpy(encoded_training_data).int().to(device)
train_tensor_encoded=train_tensor_encoded.to(torch.long)

In [10]:
# encode corpus in positional tokens Train data Dummy comparison
encoded_training_data_dummy = np.array([[token_to_index[token] for token in sequence] for sequence in np.array(train_dummy_trials_corpus_df)])
train_dummy_tensor_encoded=torch.from_numpy(encoded_training_data_dummy).int().to(device)
train_dummy_tensor_encoded=train_dummy_tensor_encoded.to(torch.long)

In [11]:
# set seed for reproductibility
torch.manual_seed(183)

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 4 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200

n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [12]:
# here are all the unique characters that occur in this text
chars = tokens_list
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: [itos[i] for i in l]#lambda l: ','.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [13]:
# data loading
def get_batch(data):
    # data must be train trials encoded and transformed in torch tensor
    # generate a small batch of data of inputs x and targets y
    # data = train_tensor_encoded #if split == 'train' else val_data
    ix = torch.randint(len(data), (batch_size,)) 
    x = torch.stack([data[i,:block_size] for i in ix])
    y = torch.stack([data[i,1:] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    # data = train_tensor_encoded# Modify for train several models
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
        X, Y = get_batch(data)
        logits, loss = model(X, Y)
        losses[k] = loss.item()
    out["train"] = losses.mean()
    model.train()
    return out

The next cell defines if the model is a decoder or encoder block.

line 21 `wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))` adds the mask filter in the GPT decoder block

remove this mask filter and you get the Encoder (BERT) block.

In [14]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        # wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T) #filter communication with past# comment this and you have an encoder block
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [15]:
# I created this function for call the trained model 

def get_trials_answers(tensor_encoded, model_trained, tell_me=True, tell_me_n_trials=1000):
    # tensor_encoded: Pytorch tensor long(float64) type [sample, comp_1, comp_2, comp_3, comp_response] of all group trials
    max_iters=tensor_encoded.shape[0]
    response_stimulus=[]
    for count, trial in enumerate(tensor_encoded[:,:-1]):
        tokens_response=model_trained.generate(trial.reshape([1,-1]), max_new_tokens=1)[0].tolist()
        response=decode(tokens_response)[-1]
        response_stimulus.append(response)

        if tell_me:
            # tell me the progress every tell_me_n_trials
            if count % tell_me_n_trials == 0 or count == max_iters - 1:
                print(f"{count} ({100.0*(count/max_iters):.2f}%) trials processed")
    return response_stimulus

In [16]:
########### Model_1 train trials class comparison

In [17]:
model_class = GPTLanguageModel()
m_class = model_class.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m_class.parameters())/1e6, 'M parameters')

data=train_tensor_encoded
model=m_class

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model_class.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        # print(losses)
        print(f"step {iter}: train loss {losses['train']:.4f}")

    # sample a batch of data
    xb, yb = get_batch(data)

    # evaluate the loss
    logits, loss = model_class(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

10.682164 M parameters
step 0: train loss 4.0431
step 500: train loss 0.0015
step 1000: train loss 0.0004
step 1500: train loss 0.0001
step 2000: train loss 0.0001
step 2500: train loss 0.0006
step 3000: train loss 0.0001
step 3500: train loss 0.0001
step 4000: train loss 0.0002
step 4500: train loss 0.0002
step 4999: train loss 0.0000


In [18]:
#### Predict model data
train_responses_class=get_trials_answers(train_tensor_encoded, m_class)

0 (0.00%) trials processed
1000 (3.31%) trials processed
2000 (6.61%) trials processed
3000 (9.92%) trials processed
4000 (13.23%) trials processed
5000 (16.53%) trials processed
6000 (19.84%) trials processed
7000 (23.15%) trials processed
8000 (26.46%) trials processed
9000 (29.76%) trials processed
10000 (33.07%) trials processed
11000 (36.38%) trials processed
12000 (39.68%) trials processed
13000 (42.99%) trials processed
14000 (46.30%) trials processed
15000 (49.60%) trials processed
16000 (52.91%) trials processed
17000 (56.22%) trials processed
18000 (59.52%) trials processed
19000 (62.83%) trials processed
20000 (66.14%) trials processed
21000 (69.44%) trials processed
22000 (72.75%) trials processed
23000 (76.06%) trials processed
24000 (79.37%) trials processed
25000 (82.67%) trials processed
26000 (85.98%) trials processed
27000 (89.29%) trials processed
28000 (92.59%) trials processed
29000 (95.90%) trials processed
30000 (99.21%) trials processed
30239 (100.00%) trials pr

In [19]:
########### Model_2 train trials dummy comparison

In [20]:
model_dummy = GPTLanguageModel()
m_dummy = model_dummy.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m_dummy.parameters())/1e6, 'M parameters')

data=train_dummy_tensor_encoded
model=m_dummy

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model_dummy.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        # print(losses)
        print(f"step {iter}: train loss {losses['train']:.4f}")

    # sample a batch of data
    xb, yb = get_batch(data)

    # evaluate the loss
    logits, loss = model_dummy(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

10.682164 M parameters
step 0: train loss 4.0001
step 500: train loss 0.0009
step 1000: train loss 0.0003
step 1500: train loss 0.0001
step 2000: train loss 0.0001
step 2500: train loss 0.0001
step 3000: train loss 0.0001
step 3500: train loss 0.0001
step 4000: train loss 0.0001
step 4500: train loss 0.0001
step 4999: train loss 0.0000


In [21]:
# ### Predict model data
train_dummy_responses = get_trials_answers(train_dummy_tensor_encoded, m_dummy)

0 (0.00%) trials processed
1000 (3.31%) trials processed
2000 (6.61%) trials processed
3000 (9.92%) trials processed
4000 (13.23%) trials processed
5000 (16.53%) trials processed
6000 (19.84%) trials processed
7000 (23.15%) trials processed
8000 (26.46%) trials processed
9000 (29.76%) trials processed
10000 (33.07%) trials processed
11000 (36.38%) trials processed
12000 (39.68%) trials processed
13000 (42.99%) trials processed
14000 (46.30%) trials processed
15000 (49.60%) trials processed
16000 (52.91%) trials processed
17000 (56.22%) trials processed
18000 (59.52%) trials processed
19000 (62.83%) trials processed
20000 (66.14%) trials processed
21000 (69.44%) trials processed
22000 (72.75%) trials processed
23000 (76.06%) trials processed
24000 (79.37%) trials processed
25000 (82.67%) trials processed
26000 (85.98%) trials processed
27000 (89.29%) trials processed
28000 (92.59%) trials processed
29000 (95.90%) trials processed
30000 (99.21%) trials processed
30239 (100.00%) trials pr